# SPARQL Queries to retrieve data from the constructed RDF graph


In [ ]:
# Import necessary libraries
# !pip install rdflib # uncomment if necessary
import rdflib
from rdflib import Graph, URIRef, Namespace, SDO
import pandas as pd

# Load the graph (assuming it's saved as 'food_nutrition_graph.rdf')
BASE = Namespace("http://kg-course.io/food-nutrition/")
SCHEMA = Namespace("http://kg-course.io/food-nutrition/schema/")
g = Graph()
g.bind("base", BASE)
g.bind("schema", SCHEMA)

g.parse("food_nutrition_graph.ttl", format="ttl")

prefix_string = """ @prefix ex: <http://kg-course.io/food-nutrition/schema/> .
    @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
    @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
    @prefix schema: <https://schema.org/> .
    @prefix xsd: <http://www.w3.org/2001/XMLSchema#> ."""

## Find recipes that are risky for individuals allergic to mango

In [ ]:
results1 = g.query(
    f"""
    {prefix_string}
    SELECT ?recipe ?recipeLabel WHERE {{
        ?recipe a schema:Recipe ;
            rdfs:label ?recipeLabel ;
            schema:recipeIngredient ?ingredients .
        FILTER (CONTAINS(lcase(str(?movieLabel)), "mango"))
    }}
    """
)
results1 = pd.DataFrame(results1.bindings)
results1

## List all pies tagged as 'healthy' with a cooking time of less than 2 hours

In [ ]:
results2 = g.query(
    f"""
    {prefix_string}
    SELECT ?recipe ?recipeLabel WHERE {{
        ?recipe a schema:Recipe ;
            schema:keywords ?keywords ;
            schema:recipeCategory ?recipeCategory .
        FILTER(CONTAINS(lcase(str(?recipeCategory)), "pie"))
        FILTER(CONTAINS(lcase(str(?keywords)), "pie"))
        FILTER(CONTAINS(lcase(str(?keywords)), "healthy"))
    }}
    """
)
results2 = pd.DataFrame(results1.bindings)
results2

## List all restaurants in New Delhi that serve Chinese cuisine and offer online delivery

## Find the average cost of two dining at restaurants in Davenport that serve Asian food

## Recommend the top 5 desserts published after 2000 that are labeled as ‘Easy’ and low in calories(< 300), along with their images

## Identify the top 10 highly rated beverages, including their preparation time and sugar content

## Identify the highest-rated recipes containing protein-rich (> 20) ingredients and check if their corresponding cuisines are commonly available in the USA restaurants

## Find the top 5 healthiest recipes based on a nutrition density score (NDS), along with their average sentiment from reviews. Check if restaurants serve cuisines linked to these recipes and retrieve their aggregate restaurant ratings.

To calculate an NDS, you will incorporate weighting factors based on dietary importance, i.e., high protein, high fiber, low sugar.

> 𝑁𝐷𝑆 = (1.0 × 𝑃𝑟𝑜𝑡𝑒𝑖𝑛(𝑔)) + (1. 5 × 𝐹𝑖𝑏𝑒𝑟(𝑔)) − (2. 0 × 𝑆𝑢𝑔𝑎𝑟(𝑔))

Protein is essential for muscle and metabolism. Fiber is crucial for digestion and satiety. Excess sugar contributes to metabolic issues, hence higher penalty